#JUPYTER FORMAT TO CHECK OUTPUT AS CODE WRITES TO .XLSX FILE

In [1]:
from riotwatcher import LolWatcher
import time

#DEFINE FUNCTIONS
def get_matchData(match_ID, player_id, player_routing, lol_watcher):
    game_data = lol_watcher.match.by_id(region = player_routing, match_id = match_ID)

    #GET TOP LEVEL Match & Team STATS
    participant_data = game_data['info']['participants']

    #TEAMS ARE 0-4 AND 5-9, SO FIND THE ID OF THE SUMMONER
    player_id = player_id
    for item in participant_data:
        if item['summonerId'] == player_id:
            p_id = game_data['info']['participants'].index(item)
        else:
            pass

    #GET INDIVIDUAL STATS
    player_stats = participant_data[p_id]

    #LANE, ROLE, W/L RATE
    lane = player_stats['lane']
    role = player_stats['role']
    win_loss = float(player_stats['win'])

    return lane, role, win_loss

if __name__ == '__main__':
    start_time = time.time()

    api_key = ['API_KEY']
    lol_watcher = LolWatcher(api_key)

    player_id = 'CwIAo4JyRAn5qpR9vElq4hR3pmSI-lfprI5Y97Q_AU4SJN99N8ycWTtI4A'
    player_region = 'NA1'.lower()
    player_routing = "americas"

    summoner = lol_watcher.summoner.by_id(player_region, player_id)
    match_history = lol_watcher.match.matchlist_by_puuid(region = player_routing, puuid = summoner['puuid'],
                                                        queue = 420,
                                                        start = 0, count = 2)
    
    last_match = match_history[0]

    [lane, role, win_loss] = get_matchData(last_match, player_id, player_routing, lol_watcher)
    print(lane)
    print(role)

    print('\n--- %s seconds ---' % (time.time() - start_time))

JUNGLE
NONE

--- 1.4040300846099854 seconds ---


In [2]:
#LOAD LIBRARIES
from riotwatcher import LolWatcher
import time

from collections import Counter
import pandas as pd
import numpy as np
from openpyxl.workbook import Workbook

#DEFINE FUNCTIONS
def setup_env():
    api_key = ['API_KEY']

    lol_watcher = LolWatcher(api_key)

    return lol_watcher

def get_match_history(summonerId, player_routing, summoner):
    while True:
        try:
            match_history = lol_watcher.match.matchlist_by_puuid(region = player_routing, puuid = summoner['puuid'],
                                                                  queue = 420, start = 0, count = 5)
            return match_history
        except Exception as e:
            if hasattr(e, 'response') and e.response.status_code == 429:  # Rate limit exceeded
                print("Rate limit exceeded. Waiting before retrying...")
                time.sleep(60)  # Sleep for 60 seconds before retrying
            else:
                print(f"Error {e.response.status_code}: {e.response.text}")
                return None

def get_challenger_players(player_region, queue_type, lol_watcher):
    while True:
        try:
            challenger_players = pd.DataFrame.from_dict(
                lol_watcher.league.challenger_by_queue(region = player_region, queue = queue_type)['entries'])
            challenger_players = challenger_players.sort_values(by = 'leaguePoints', ascending = False)
            challenger_players.reset_index(drop = True, inplace = True)
            summoner_Ids = challenger_players['summonerId'].tolist()
            return summoner_Ids
        except Exception as e:
            if hasattr(e, 'response') and e.response.status_code == 429:  # Rate limit exceeded
                print("Rate limit exceeded. Waiting before retrying...")
                time.sleep(60)  # Sleep for 60 seconds before retrying
            else:
                print(f"Error {e.response.status_code}: {e.response.text}")
                return None

def get_challenger_lane(summonerId, player_routing, lol_watcher):
    summoner = lol_watcher.summoner.by_id(player_region, summonerId)
    match_history = get_match_history(summonerId, player_region, summoner)
    
    roles_played = []
    lanes_played = []
    win_loss_list = []
    
    for matchID in match_history:
        try:
            [lane, role, win] = get_matchData(matchID, summonerId, player_routing, lol_watcher)
            roles_played.append(role)

            if role == 'DUO':
                lanes_played.append('BOTTOM_' + role)
            elif (role == 'SUPPORT' and lane == 'JUNGLE'):
                lanes_played.append(lane)
            elif role == 'SUPPORT':
                lanes_played.append('BOTTOM_' + role)
            else:
                lanes_played.append(lane)
            win_loss_list.append(win)

        except:
            pass
    
    player_pref_lane = Counter(lanes_played).most_common(1)[0][0]
    player_pref_role = Counter(roles_played).most_common(1)[0][0]
    win_loss_ratio = np.mean(win_loss_list)

    return [player_pref_lane, player_pref_role, win_loss_ratio]

#MAIN CODE
start_time = time.time()
lol_watcher = setup_env()

#GET Challenger Players
player_region = 'NA1'.lower()
player_routing = 'americas'
queue_type = 'RANKED_SOLO_5x5'

summoner_Ids = get_challenger_players(player_region, queue_type, lol_watcher)

col_names = ['lane', 'role', 'win_rate']
raw_data_list = []

for summonerID in summoner_Ids:
    print(summonerID)

    result = get_challenger_lane(summonerID, player_routing, lol_watcher)
    raw_data_list.append(result)

raw_data = pd.DataFrame(raw_data_list, columns=col_names)
raw_data['summoner ID'] = summoner_Ids

filename = player_region + '_challenger_lanes' + '.xlsx'
raw_data.to_excel(filename, index=False)

print(f"Excel file '{filename}' has been successfully created.")
print('\n--- %s seconds --- ' % (time.time() - start_time))


Xi7HdSnX94Nw2l-vv9IWBeSBinjRtvkIuhlGZCfd7BlLWWAXgT3WdOLU4Q
bLp7NFhxcoeJTaRQeTqhzFwRJK3k7_iiTfLkBiV6Uw_ratCj
wTcd_EsGUQ6qnH3fCEdKbzErPzNx1H23MH79jf_FHj5fJ28LWeMySNCRkg
We2_ia-pxHYe6zC5H7nDBIhTWZLVmLk64-wfnESDxCzE5fzx
ROvSV9CwnzlZyc0giKvNyKgCbbDUcqYlXONyu09JCB-z26LR5JhOsR7yJg
guHMnPbBTb50KBvZcnBM9I46UJ4yowma_ZM0vGpuYRctvRPkc0LakasV5A
IUB3yM2W8YzQwwc7TbRGyb5S1rZpiZ7aIB99NAesegVPmz0
NA9jmvBX8ip1j51hgwp-DFEu-XFPNZaL0bAH4D3ceRMpc1TLBo1_utyK3A
j9tdDh2iVXSFuzygVROJhQ2V08lPzbKc5Absz5zMaX-LXQjyAMy_yYO4sA
qZkJgIrVKNL0t5WaI14oGVS2fyTXepN7Oso8FWj6n4b8hY4
nEN5yJhF3CfxfBDZfNr6-xpjLZgdbpQ4i73L7jfM_r_QvNY
nQj5lvcmkPyAMbuBDEAR3l8R0makcdIHjFz7rwUkkLh2-As
N87NT23lLEQN7AtDMswiQbybLvPgdLojhNGJF4Y9f3EkTns
njRpzEfGL3yxbDziEo4oRjTaSdej2jsMlvzowsSAqkmSdRY
WuB6RWNFVU_lv5wEDtZ_J_BuX3dkJv4iWlIAONmoQwSsCj8
UnkD-vGRUK3qOztZof8QED4ho4hVdi1Y2Evhug4mV5XsLsk
gq-4xWzZe_Cj32ryMDgQDt9zOpvSniDAb_ZNo-vSg6YjLTY
a45gdEB4xlwbFlL0chqc3iqt-pL4BdNdOHhSof7Ix8XOv9Y
FqHR_XhbbwsKl6PJXBW0hs8D_7ytVp0NyGFhaAGIJwjmCz2T
hNGsp6hCm03kDmehE4A